**In this guide we will demonstrate how to use Nano AutoML to auto tune an machine learning model and handle the whole model development process easily.**

# Introduction
Nano provides hpo as an solution for fast and high-performance hyperparameter tuning. Users can import hano hpo following the standard tensorflow API without changing their own code. By setting the search space and running model.search(), the model will contain the best hyperparameters.

Nano hpo supports sequential, fuctional and custom keras API.


# Step0: Prepare Environment
We recommend using conda to prepare the environment.
conda create -n my_env python=3.7

conda activate my_env

pip install --pre --upgrade bigdl

# Step1: Init Nano AutoML
Let's start out by importing Nano AutoML and enable its tensorflow API globally by automl.hpo_config.enable_hpo_tf().

After importing, we will load MNIST as our data.

In [1]:
#to do: remove
import sys
sys.path.append("../../src/")

import bigdl.nano.automl as automl
automl.hpo_config.enable_hpo_tf()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import RMSprop

import bigdl.nano.automl.hpo as hpo


N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
CLASSES = 10

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
img_x, img_y = x_train.shape[1], x_train.shape[2]
x_train = x_train.reshape(-1, img_x, img_y,
                          1)[:N_TRAIN_EXAMPLES].astype("float32") / 255
x_test = x_test.reshape(-1, img_x, img_y,
                          1)[:N_VALID_EXAMPLES].astype("float32") / 255
y_train = y_train[:N_TRAIN_EXAMPLES]
y_test = y_test[:N_VALID_EXAMPLES]
input_shape = (img_x, img_y, 1)

../../src/bigdl/nano/tf/__init__.py:24: UserWarning: NANO_TF_INTER_OP not found the in os.environ, please run `source bigdl-nano-init`
  warnings.warn("NANO_TF_INTER_OP not found the in os.environ, "
../../src/bigdl/nano/tf/__init__.py:30: UserWarning: OMP_NUM_THREADS not found the in os.environ, please run `source bigdl-nano-init`
  warnings.warn("OMP_NUM_THREADS not found the in os.environ, "


# Step2: Set search space
We now create our model function. 

Whenever you want to search a hyperparameter, you can simply replace it with a space and set your own searching space 


In [2]:
@hpo.tfmodel()
class MyModel(tf.keras.Model):

    def __init__(self, filters, kernel_size, strides, activation):
        super().__init__()
        self.conv1 = Conv2D(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            activation=activation)
        self.flat = Flatten()
        self.dense = Dense(CLASSES, activation="softmax")

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.flat(x)
        x = self.dense(x)
        return x
model = MyModel(
    filters=hpo.space.Categorical(32, 64),
    kernel_size=hpo.space.Categorical(3, 5),
    strides=hpo.space.Categorical(1, 2),
    activation=hpo.space.Categorical("relu", "linear")
)
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=RMSprop(learning_rate=1e-4),
    metrics=["accuracy"],
)

2022-05-18 15:35:34.724240: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-18 15:35:34.725306: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 


# Step3: Apply searching
Apply hyperparameter searching by calling model.search(), and best hyperparameters will be saved in the model.

You can specify the number of trial, each trial will use a set of hyperparameters automatiacally suggusted by nano.hpo. 

You can set your target metric and whehter you want to maximize you minimize it.

In [3]:
%%time
model.search(
    n_trials=20,
    target_metric='accuracy',
    direction="maximize",
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=24,
    validation_split=0.2,
    verbose=False,
)

[I 2022-05-18 15:35:34,738] A new study created in memory with name: no-name-a18ae430-a7f7-4b6d-9cb5-e209bb351ab5


Starting a new tuning
guoyao want to know compile args
() {'loss': 'sparse_categorical_crossentropy', 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7fcaf002ea90>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
19/19 - 1s - loss: 2.2644 - accuracy: 0.2037 - val_loss: 2.2266 - val_accuracy: 0.4267 - 580ms/epoch - 31ms/step
Epoch 2/24
19/19 - 0s - loss: 2.1906 - accuracy: 0.5471 - val_loss: 2.1555 - val_accuracy: 0.6500 - 204ms/epoch - 11ms/step
Epoch 3/24
19/19 - 0s - loss: 2.1158 - accuracy: 0.7108 - val_loss: 2.0775 - val_accuracy: 0.7117 - 174ms/epoch - 9ms/step
Epoch 4/24
19/19 - 0s - loss: 2.0341 - accuracy: 0.7588 - val_loss: 1.9924 - val_accuracy: 0.7467 - 193ms/epoch - 10ms/step
Epoch 5/24
19/19 - 0s - loss: 1.9459 - accuracy: 0.7688 - val_loss: 1.9019 - val_accuracy: 0.7600 - 253ms/epoch - 13ms/step
Epoch 6/24
19/19 - 0s - loss: 1.8522 - accuracy: 0.7821 - val_loss: 1.8057 - val_accuracy: 0.7650 - 225ms/epoch - 12ms/step
Epoch 7/24
19/19 - 0s - loss: 1.7539 - ac

[I 2022-05-18 15:35:45,328] Trial 0 finished with value: 0.8695833086967468 and parameters: {'activation▁choice': 0, 'filters▁choice': 0, 'kernel_size▁choice': 0, 'strides▁choice': 1}. Best is trial 0 with value: 0.8695833086967468.


guoyao want to know compile args
() {'loss': 'sparse_categorical_crossentropy', 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7fcac86d1150>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
19/19 - 1s - loss: 1.9020 - accuracy: 0.6350 - val_loss: 1.5420 - val_accuracy: 0.7933 - 1s/epoch - 60ms/step
Epoch 2/24
19/19 - 1s - loss: 1.3137 - accuracy: 0.8108 - val_loss: 1.0820 - val_accuracy: 0.8117 - 667ms/epoch - 35ms/step
Epoch 3/24
19/19 - 1s - loss: 0.9358 - accuracy: 0.8404 - val_loss: 0.7978 - val_accuracy: 0.8350 - 726ms/epoch - 38ms/step
Epoch 4/24
19/19 - 1s - loss: 0.7065 - accuracy: 0.8579 - val_loss: 0.6371 - val_accuracy: 0.8500 - 719ms/epoch - 38ms/step
Epoch 5/24
19/19 - 1s - loss: 0.5696 - accuracy: 0.8679 - val_loss: 0.5404 - val_accuracy: 0.8617 - 714ms/epoch - 38ms/step
Epoch 6/24
19/19 - 1s - loss: 0.4824 - accuracy: 0.8817 - val_loss: 0.4679 - val_accuracy: 0.8717 - 672ms/epoch - 35ms/step
Epoch 7/24
19/19 - 1s - loss: 0.4230 - accuracy: 0.8900 - val_los

[I 2022-05-18 15:36:01,723] Trial 1 finished with value: 0.9495833516120911 and parameters: {'activation▁choice': 0, 'filters▁choice': 1, 'kernel_size▁choice': 1, 'strides▁choice': 0}. Best is trial 1 with value: 0.9495833516120911.


...

guoyao want to know compile args
() {'loss': 'sparse_categorical_crossentropy', 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7fcae28fb5d0>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
19/19 - 1s - loss: 1.6927 - accuracy: 0.6775 - val_loss: 1.2791 - val_accuracy: 0.7867 - 1s/epoch - 54ms/step
Epoch 2/24
19/19 - 1s - loss: 1.0838 - accuracy: 0.8112 - val_loss: 0.9073 - val_accuracy: 0.8200 - 624ms/epoch - 33ms/step
Epoch 3/24
19/19 - 1s - loss: 0.7962 - accuracy: 0.8404 - val_loss: 0.7026 - val_accuracy: 0.8383 - 622ms/epoch - 33ms/step
Epoch 4/24
19/19 - 0s - loss: 0.6283 - accuracy: 0.8612 - val_loss: 0.5829 - val_accuracy: 0.8567 - 450ms/epoch - 24ms/step
Epoch 5/24
19/19 - 1s - loss: 0.5251 - accuracy: 0.8717 - val_loss: 0.5133 - val_accuracy: 0.8550 - 551ms/epoch - 29ms/step
Epoch 6/24
19/19 - 0s - loss: 0.4553 - accuracy: 0.8879 - val_loss: 0.4533 - val_accuracy: 0.8783 - 462ms/epoch - 24ms/step
Epoch 7/24
19/19 - 0s - loss: 0.4057 - accuracy: 0.8954 - val_los

[I 2022-05-18 15:39:45,745] Trial 18 finished with value: 0.9504166841506958 and parameters: {'activation▁choice': 1, 'filters▁choice': 1, 'kernel_size▁choice': 0, 'strides▁choice': 0}. Best is trial 18 with value: 0.9504166841506958.


guoyao want to know compile args
() {'loss': 'sparse_categorical_crossentropy', 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7fcae3372550>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
19/19 - 1s - loss: 1.6927 - accuracy: 0.6775 - val_loss: 1.2791 - val_accuracy: 0.7867 - 856ms/epoch - 45ms/step
Epoch 2/24
19/19 - 1s - loss: 1.0838 - accuracy: 0.8112 - val_loss: 0.9073 - val_accuracy: 0.8200 - 638ms/epoch - 34ms/step
Epoch 3/24
19/19 - 1s - loss: 0.7962 - accuracy: 0.8404 - val_loss: 0.7026 - val_accuracy: 0.8383 - 692ms/epoch - 36ms/step
Epoch 4/24
19/19 - 1s - loss: 0.6283 - accuracy: 0.8612 - val_loss: 0.5829 - val_accuracy: 0.8567 - 544ms/epoch - 29ms/step
Epoch 5/24
19/19 - 0s - loss: 0.5251 - accuracy: 0.8717 - val_loss: 0.5133 - val_accuracy: 0.8550 - 407ms/epoch - 21ms/step
Epoch 6/24
19/19 - 0s - loss: 0.4553 - accuracy: 0.8879 - val_loss: 0.4533 - val_accuracy: 0.8783 - 412ms/epoch - 22ms/step
Epoch 7/24
19/19 - 0s - loss: 0.4057 - accuracy: 0.8954 - val_

[I 2022-05-18 15:39:58,507] Trial 19 finished with value: 0.9504166841506958 and parameters: {'activation▁choice': 1, 'filters▁choice': 1, 'kernel_size▁choice': 0, 'strides▁choice': 0}. Best is trial 18 with value: 0.9504166841506958.


CPU times: user 18min 5s, sys: 13min 26s, total: 31min 31s
Wall time: 4min 23s


# Step 4: fit with the best hyperparameters
The best hyperparameters have been saved in the model. You can call model.fit() to train a model with the best hyperparameters.

In [4]:
history = model.fit(x_train, y_train,
                    batch_size=128, epochs=24, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

guoyao want to know compile args
() {'loss': 'sparse_categorical_crossentropy', 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7fcae810b1d0>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
19/19 [==============================] - 1s 35ms/step - loss: 1.6927 - accuracy: 0.6775 - val_loss: 1.2791 - val_accuracy: 0.7867
Epoch 2/24
19/19 [==============================] - 0s 22ms/step - loss: 1.0838 - accuracy: 0.8112 - val_loss: 0.9073 - val_accuracy: 0.8200
Epoch 3/24
19/19 [==============================] - 0s 22ms/step - loss: 0.7962 - accuracy: 0.8404 - val_loss: 0.7026 - val_accuracy: 0.8383
Epoch 4/24
19/19 [==============================] - 0s 22ms/step - loss: 0.6283 - accuracy: 0.8612 - val_loss: 0.5829 - val_accuracy: 0.8567
Epoch 5/24
19/19 [==============================] - 1s 27ms/step - loss: 0.5251 - accuracy: 0.8717 - val_loss: 0.5133 - val_accuracy: 0.8550
Epoch 6/24
19/19 [==============================] - 1s 29ms/step - loss: 0.4553 - accuracy: 0.8879 - 

Check out the summary of the model. The model has already been built with the best hyperparameters found by nano hpo.

In [5]:
print(model.summary())

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           multiple                  640       
                                                                 
 flatten_1 (Flatten)         multiple                  0         
                                                                 
 dense_1 (Dense)             multiple                  432650    
                                                                 
Total params: 433,290
Trainable params: 433,290
Non-trainable params: 0
_________________________________________________________________
None
